## RESNET152 model for training Food101 datasets
* number of parameters: 58,350,757

In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets

from torchinfo import summary

from pathlib import Path

import sys
sys.path.append('..')

from python_scripts import model_builder, engine, data_setup

In [10]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [11]:
food101_transforms_train = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

food101_transforms_test = transforms.Compose([
    transforms.Resize(225),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

In [12]:
learning_rate_list = [1e-3]
weight_decay_list = [1e-4]
epochs_list = [5]
batch_size_list = [16]

In [13]:
data_dir = Path('../data/extracted')

train_data_food101 = datasets.Food101(
    root=data_dir,
    split='train',
    transform=food101_transforms_train,
    download=True
)
test_data_food101 = datasets.Food101(
    root=data_dir,
    split='test',
    transform=food101_transforms_test,
    download=True
)

class_names_food101 = train_data_food101.classes

In [14]:
def resnet_model_generator(weights=None):
    return model_builder.Resnet152(in_channels=3, num_classes=len(class_names_food101))

In [15]:
train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_food101,
    split_size=1, # you can start with small number of train set to check normality at first
    seed=42
)
test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.HP_tune_train(
    model=None,
    model_generator=resnet_model_generator,
    model_weights=None,
    model_name='ResNet158_food101',
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=False,
    device=device,
    gradient_accumulation_num=1
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 4.6478, Train_acc: 0.0154 | Test_loss: 4.4987, Test_acc: 0.0248
Epoch: 1 | Train_loss: 4.4524, Train_acc: 0.0324 | Test_loss: 4.2873, Test_acc: 0.0456
Epoch: 2 | Train_loss: 4.3646, Train_acc: 0.0425 | Test_loss: 4.1518, Test_acc: 0.0718
Epoch: 3 | Train_loss: 4.2968, Train_acc: 0.0532 | Test_loss: 4.0794, Test_acc: 0.0734
Epoch: 4 | Train_loss: 4.2406, Train_acc: 0.0621 | Test_loss: 3.9423, Test_acc: 0.1041


In [ ]:
import pandas as pd
pd.DataFrame(tuning_results)